In [11]:
import requests
import numpy as np
import random

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def get_ans():
	'''Asks user for input. If the user types yes true is returned'''
	ans = input()
	if ans.lower == 'y' or ans.lower == 'yes' or ans == '1':
		return True
	else:
		return False

def get_wcif(comp):
	'''Pulls WCIF from WCA and return json object. If there is a problem with the request the program terminates'''
	print('Attempting to pull competition info...')
	# Send GET request for WCIF
	WCIF = requests.get('https://www.worldcubeassociation.org/api/v0/competitions/' + comp + '/wcif/public')
	# If response code is 200 everything good, otherwise something bad
	if WCIF.status_code == 200:
		print('Success!')
		return WCIF.json()
	else:
		print('Response Error!')
		exit()
        
        
# Dictionaries to translate event ID to event name
# First element: LaTeX. Second element: plain text.
event_dict = {
	'222': ['2$\\times$2$\\times$2 Cube', '2x2x2 Cube'],
	'333': ['3$\\times$3$\\times$3 Cube', '3x3x3 Cube'],
	'444': ['4$\\times$4$\\times$4 Cube', '4x4x4 Cube'],
	'555': ['5$\\times$5$\\times$5 Cube', '5x5x5 Cube'],
	'666': ['6$\\times$6$\\times$6 Cube', '6x6x6 Cube'],
	'777': ['7$\\times$7$\\times$7 Cube', '7x7x7 Cube'],
	'333bf': ['3$\\times$3$\\times$3 Blindfolded', '3x3x3 Blindfolded'],
	'333oh': ['3$\\times$3$\\times$3 One-Handed', '3x3x3 One-Handed'],
	'333fm': ['3$\\times$3$\\times$3 Fewest Moves', '3x3x3 Fewest Moves'],
	'333mbf': ['3$\\times$3$\\times$3 Multi-Blindfolded', '3x3x3 Multi-Blindfolded'],
	'minx': ['Megaminx', 'Megaminx'],
	'clock': ['Clock', 'Clock'],
	'pyram': ['Pyraminx', 'Pyraminx'],
	'skewb': ['Skewb', 'Skewb'],
	'sq1': ['Square-1', 'Square-1'],
	'444bf': ['4$\\times$4$\\times$4 Blindfolded', '4x4x4 Blindfolded'],
	'555bf': ['5$\\times$5$\\times$5 Blindfolded', '5x5x5 Blindfolded']
}

In [4]:
# Get comp name from user
print('Enter competition ID:')
#comp = input()
comp = 'Cubinginthe6ix2019'
#comp = 'NA2020'

WCIF = get_wcif(comp)

Enter competition ID:
Attempting to pull competition info...
Success!


In [73]:
class Person:
    def __init__(self, name, ID, events, wcaid, role, num_events):
        '''Person constructor'''
        self.name = name
        self.id = ID
        self.events = events
        self.wcaid = wcaid
        self.role = role
        self.groups = [''] * num_events
        
    @staticmethod
    def build_persons(WCIF):
        '''Builds an array of competitors from WCIF data'''
        lst = []
        i = 0 # Counter
        for pers in WCIF['persons']: # Loops over each person registered
            # Build competitors
            if pers['registration'] != None:
                if 'delegate' in pers['roles']:
                    role = 'delegate'
                elif 'organizer' in pers['roles']:
                    role = 'organizer'
                else:
                    role = 'competitor'
                lst.append(Person(pers['name'], pers['registrantId'], pers['registration']['eventIds'], pers['wcaId'], role, len(WCIF['events'])))
                if lst[i].wcaid == None:
                    lst[i].wcaid = ''
                i += 1
        return lst
        
        
class Competition:    
    def event_group(self, eventid, index, g_size = 16):
        # Extracts persons competing in eventid
        competing = [i for i in self.competitors if eventid in i.events]
        random.shuffle(competing) # Randomized groups

        N = len(competing) # Number of competitors in event
        num_g = int(np.round(N / float(g_size))) # Number of groups

        count = 0
        for pers in competing:
            if eventid in ['333fm', '333mbf', '444bf', '555bf']: # 1 group for long events
                pers.groups[index] = '1'
                count += 1
            else:
                pers.groups[index] = str(int(count * num_g / N) + 1) # Record group
                count += 1            
            
    def group(self):
        for i in range(len(self.events)):
            self.event_group(self.events[i], i)
            
    def write_nametags(self, f_name, num_blank = 0):
        self.competitors.sort(key=lambda i: i.name) # Sort by name
        f = open(f_name, 'w')
        for pers in self.competitors:
            f.write('\\nametag{%s}{%s}{%s}%%\n' % (pers.name, pers.role.upper(), pers.wcaid))
        for i in range(num_blank): # Include blank nametags for day-of registrations
            f.write('\\nametag{}{COMPETITOR}{}%\n')
        f.close()
        
    def write_tex_groups(self, f_name):
        self.competitors.sort(key = lambda i: i.name) # Sort by name
        f = open(f_name, 'w')
        for pers in self.competitors:
            f.write('\\groups{%s}{' % (pers.name))
            for i in range(len(self.events)):
                f.write('%s & %s \\\\ ' % (event_dict[self.events[i]][0], pers.groups[i]))
            f.write('}% \n')
        f.close()
        
    def write_wca_groups(self, f_name):
        self.competitors.sort(key = lambda i: i.name) # Sort by name
        f = open(f_name, 'w')
        # Header
        f.write('| Name |')
        for i in self.events:
            f.write(' %s |' % (event_dict[i][1]))
        f.write('\n|' + ' --- |' * (1 + len(self.events)) + '\n')
        # Groups
        for pers in self.competitors:
            f.write('| %s |' % (pers.name))
            for i in range(len(self.events)):
                f.write(' %s |' % (pers.groups[i]))
            f.write('\n')
        f.close()
              
    @staticmethod
    def centi2min(centi):
        '''Converts a time in centiseconds to minute:second format'''
        minute = int(np.floor(centi / 6000)) # Compute number of minutes
        sec = int(np.ceil((centi - 6000 * minute) / 100)) # Compute remainder
        return str(minute) + ':' + str(sec).zfill(2) # Format string (zfill zero pads)
    
    @staticmethod
    def build_events(WCIF):
        '''Build a list of events held at the competition'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            lst[i] = WCIF['events'][i]['id']
        return lst
    
    @staticmethod
    def build_cutoffs(WCIF):
        '''Build a list of cutoffs for the competition. Note: Assumes cutoffs only in the first round'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['cutoff'] == None:
                lst[i] = ''
            else:
                lst[i] = Competition.centi2min(WCIF['events'][i]['rounds'][0]['cutoff']['attemptResult'])
        return lst
    
    @staticmethod
    def build_limits(WCIF):
        '''Build a list of time limits for the competition'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['cutoff'] == None:
                lst[i] = ''
            else:
                lst[i] = Competition.centi2min(WCIF['events'][i]['rounds'][0]['timeLimit']['centiseconds'])
        return lst
    
    @staticmethod
    def build_rounds(WCIF):
        '''Build a list of additional rounds for the competition'''
        lst = [[] for i in range(len(WCIF['events']))]
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['advancementCondition'] != None:
                for j in range(len(WCIF['events'][i]['rounds']) - 1):
                    lst[i].append(WCIF['events'][i]['rounds'][j]['advancementCondition']['level'])
        return lst
    
    
    
    def write_scorecards(self, f_name, num_blank = 0):
        f = open(f_name, 'w')
        
        # First round cards
        count = 0
        for event in self.events:
            if event != '333fm':
                self.competitors.sort(key = lambda i: (i.groups[count], i.name)) # Sort by group then by name
                for pers in [i for i in self.competitors if event in i.events]:
                    if event in ['666', '777', '333bf', '333mbf', '444bf', '555bf']:
                        f.write('\scorecard[1]')
                    else:
                        f.write('\scorecard')
                    f.write('{%s}{%s}{%s}{%s}{1}{%s}%%\n' % (pers.name, str(pers.id), event_dict[event][0], self.cutoffs[count], pers.groups[count]))
            count += 1
            f.write('\pagereset\n')
            
        # Subsequent round cards
        count = 0
        for event in self.events:
            if event != '333fm':
                for i in range(len(self.rounds[count])): # Round number
                    for j in range(self.rounds[count][i]): # card number
                        if event in ['666', '777', '333bf', '333mbf', '444bf', '555bf']:
                            f.write('\scorecard[1]')
                        else:
                            f.write('\scorecard')
                        f.write('{}{}{%s}{}{%s}{}%%\n' % (event_dict[event][0], str(i + 2)))
                    f.write('\pagereset\n')
                count += 1
                
        # Blank cards
        for i in range(num_blank):
            f.write('\\scorecard{}{}{}{}{}{}%\n')
        f.write('\pagereset\n')
        
        f.close()
        
    def write_tex(self, f_name, url = ''):
        f = open(f_name, 'w')

        f.write('\documentclass[fast]{scorecard}\n\n\comp{%s}\n\\url{%s}\n' % (self.name, url))
        f.write('\events{')
        for event in self.events:
            f.write('\img{./Icons/%s} ' % (event))
        f.write('}\n\n')

        f.write('\\begin{document}\n\sffamily\n\centering\n\n')

        if 1:
            f.write('\input{%sNameTags}\n\pagereset\n\n' % (self.id))

        if 1:
            f.write('\input{%sGroups}\n\pagereset\n\n' % (self.id))

        if 1:
            f.write('\\newgeometry{margin=0in}\n\crosssize{0mm}\n\n')
            f.write('\input{%sCards}\n\pagereset\n\n' % (self.id))

        f.write('\end{document}')
        f.close()

        
    def __init__(self, WCIF):
        '''Competition constructor'''
        self.name = WCIF['name']
        self.id = WCIF['id']
        self.events = self.build_events(WCIF)
        self.cutoffs = self.build_cutoffs(WCIF)
        self.limits = self.build_limits(WCIF)
        self.rounds = self.build_rounds(WCIF)
        self.competitors = Person.build_persons(WCIF)
            

In [74]:
comp = Competition(WCIF)

In [75]:
comp.write_nametags(comp.id + 'NameTags.tex')
comp.group()
comp.write_tex_groups(comp.id + 'Groups.tex')
comp.write_wca_groups('WCAGroups.md')
comp.write_scorecards(comp.id + 'Cards.tex')
comp.write_tex(comp.id + '.tex', 'wcalive')

In [65]:
print(comp.events)

['333', '444', '666', '777', '333bf']


In [53]:
for p in WCIF['persons']:
    print(p['roles'])

[]
['delegate', 'organizer']
['organizer']
['staff-scrambler', 'staff-runner']
['staff-runner']
['organizer']
['delegate']
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['staff-runner']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['staff-scrambler', 'staff-runner']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
